### 9.1 Wrapper

In [1]:
import time
from functools import wraps

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [3]:
@timeit
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        
countdown(100000)
countdown(1000)
        

countdown 0.013993024826049805
countdown 0.00011014938354492188


In [4]:
countdown.__name__, countdown.__doc__, countdown.__annotations__, countdown.__wrapped__

('countdown',
 '\n    Counts down\n    ',
 {'n': int},
 <function __main__.countdown(n: int)>)

In [5]:
from inspect import signature

print(signature(countdown))

(n: int)


### 9.4 Decorator with arguments

In [6]:
import logging
logging.basicConfig(level=logging.WARNING)

In [7]:
from functools import partial

# Utility decorator to attach a func as an attribute of an obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj) # This is the wrapper around the func -> attach_wrapper(obj)(func)
    setattr(obj, func.__name__, func)
    return func

In [8]:
def logged(level, name=None, message=None):
    
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
        
    return decorate

In [9]:
@logged(logging.WARNING)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [10]:
spam()
add(3, 4)

CRITICAL:example:spam


Spam!


7

In [11]:
spam.set_level(logging.ERROR)
spam.set_message('Custom msg')
spam()

ERROR:example:Custom msg


Spam!
